<a href="https://www.kaggle.com/code/cozyhn/autogluon-baseline-ps-s3-e17?scriptVersionId=133713162" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Baseline model use autogluon

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
warnings.filterwarnings('ignore')

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        pass
#         print(os.path.join(dirname, filename))
print("done")


done


In [2]:
!cp -r /kaggle/input/autogluon-package/* ./

In [3]:
!pip install -q --no-index --find-links='/kaggle/input/autogluon-package/' 'autogluon.tabular-0.7.0-py3-none-any.whl'
print("done")

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.5.0 requires botocore<1.29.77,>=1.29.76, but you have botocore 1.29.133 which is incompatible.
momepy 0.6.0 requires shapely>=2, but you have shapely 1.8.5.post1 which is incompatible.
ydata-profiling 4.1.2 requires scipy<1.10,>=1.4.1, but you have scipy 1.10.1 which is incompatible.
done


In [4]:
!pip install -q ray==2.2.0
print("done")

done


In [5]:
from autogluon.tabular import TabularPredictor, TabularDataset

In [6]:
import random
import os
import pandas as pd
import numpy as np

### add oringin data

In [7]:
train = pd.read_csv('../input/playground-series-s3e17/train.csv')
test = pd.read_csv('../input/playground-series-s3e17/test.csv')
origin = pd.read_csv('../input/machine-failure-predictions/machine failure.csv')
submission = pd.read_csv('../input/playground-series-s3e17/sample_submission.csv')
train['generate'] = 1
test['generate'] = 0
origin['generate'] = 0

In [8]:
train.describe(include='all')

,id,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF,generate
count,136429.000000,136429,136429,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000,136429.0
unique,NaN,9976,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,L53257,L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,139,95354,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,68214.000000,NaN,NaN,299.862776,309.941070,1520.331110,40.348643,104.408901,0.015744,0.001554,0.005160,0.002397,0.003958,0.002258,1.0
std,39383.804275,NaN,NaN,1.862247,1.385173,138.736632,8.502229,63.965040,0.124486,0.039389,0.071649,0.048899,0.062789,0.047461,0.0
min,0.000000,NaN,NaN,295.300000,305.800000,1181.000000,3.800000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
25%,34107.000000,NaN,NaN,298.300000,308.700000,1432.000000,34.600000,48.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
50%,68214.000000,NaN,NaN,300.000000,310.000000,1493.000000,40.400000,106.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
75%,102321.000000,NaN,NaN,301.200000,310.900000,1580.000000,46.100000,159.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0


In [9]:
origin.describe(include='all')

,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF,generate
count,10000.00000,10000,10000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.0
unique,NaN,10000,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,M14860,L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,1,6000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,5000.50000,NaN,NaN,300.004930,310.005560,1538.776100,39.986910,107.951000,0.033900,0.004600,0.011500,0.009500,0.009800,0.00190,0.0
std,2886.89568,NaN,NaN,2.000259,1.483734,179.284096,9.968934,63.654147,0.180981,0.067671,0.106625,0.097009,0.098514,0.04355,0.0
min,1.00000,NaN,NaN,295.300000,305.700000,1168.000000,3.800000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0
25%,2500.75000,NaN,NaN,298.300000,308.800000,1423.000000,33.200000,53.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0
50%,5000.50000,NaN,NaN,300.100000,310.100000,1503.000000,40.100000,108.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0
75%,7500.25000,NaN,NaN,301.500000,311.100000,1612.000000,46.800000,162.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0


In [10]:
test.describe(include='all')

,id,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],TWF,HDF,PWF,OSF,RNF,generate
count,90954.000000,90954,90954,90954.000000,90954.000000,90954.000000,90954.000000,90954.000000,90954.000000,90954.000000,90954.000000,90954.00000,90954.000000,90954.0
unique,NaN,9909,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,L56257,L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,98,63438,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,181905.500000,NaN,NaN,299.859493,309.939375,1520.528179,40.335191,104.293962,0.001473,0.005343,0.002353,0.00387,0.002309,0.0
std,26256.302529,NaN,NaN,1.857562,1.385296,139.970419,8.504683,63.871092,0.038355,0.072903,0.048449,0.06209,0.047995,0.0
min,136429.000000,NaN,NaN,295.300000,305.700000,1168.000000,3.800000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0
25%,159167.250000,NaN,NaN,298.300000,308.700000,1432.000000,34.600000,48.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0
50%,181905.500000,NaN,NaN,300.000000,310.000000,1493.000000,40.500000,106.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0
75%,204643.750000,NaN,NaN,301.200000,310.900000,1579.000000,46.200000,158.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0


### Drop useless feature

In [11]:
train = train.drop(['id'], axis=1)
test = test.drop(['id'], axis=1)
train = train.drop(['Product ID'], axis=1)
test = test.drop(['Product ID'], axis=1)
origin = origin.drop(['UDI'], axis=1)
origin = origin.drop(['Product ID'], axis=1)

In [12]:
train = pd.concat([origin, train])

### Drop duplicate feature and check droped feature

In [13]:
duplicated_rows = train[train.duplicated(keep=False)]
duplicated_rows = duplicated_rows.sort_values('Air temperature [K]')
print("Duplicated rows:")
print(duplicated_rows)
print("--------------------------------------")
print("len duplicated_rows :",len(duplicated_rows))
print("--------------------------------------")

train = train.drop_duplicates()

# print("\n\nDataFrame after removing duplicates:")
# print(train)
train = train.reset_index(drop=True)

Duplicated rows:
       Type  Air temperature [K]  Process temperature [K]  \
86770     M                295.5                    306.1   
31169     M                295.5                    305.9   
130644    M                295.5                    306.1   
40817     M                295.5                    306.1   
40026     M                295.5                    305.9   
...     ...                  ...                      ...   
77000     L                304.2                    313.4   
23581     M                304.2                    313.4   
21231     M                304.2                    313.4   
10769     L                304.4                    313.6   
78564     L                304.4                    313.6   

        Rotational speed [rpm]  Torque [Nm]  Tool wear [min]  Machine failure  \
86770                     1669         29.0              164                0   
31169                     1518         40.6              178                0   
130644 

In [14]:
#this idea by https://www.kaggle.com/code/reymaster/ps3-17-0-96771-eda-feature-engineering-xgboost
# https://www.kaggle.com/code/reymaster/0-9713-eda-feature-engineering-ensemble-baseline
train["Power"] = train["Torque [Nm]"] * train["Rotational speed [rpm]"]
test["Power"] = test["Torque [Nm]"] * train["Rotational speed [rpm]"]

train["temp_ratio"] = train["Process temperature [K]"] / train["Air temperature [K]"]
test["temp_ratio"] = test["Process temperature [K]"] / test["Air temperature [K]"]

train["Failure Sum"] = (train["TWF"] +train["HDF"] +train["PWF"] + train["OSF"] +train["RNF"])
test["Failure Sum"] = (test["TWF"] +test["HDF"] +test["PWF"] + test["OSF"] +test["RNF"])

train["tool_wear_speed"] = train["Tool wear [min]"] * train["Rotational speed [rpm]"]
test["tool_wear_speed"] = test["Tool wear [min]"] * test["Rotational speed [rpm]"]

train["torque wear ratio"] = train["Torque [Nm]"] / (train["Tool wear [min]"] + 0.0001)
test["torque wear ratio"] = test["Torque [Nm]"] / (test["Tool wear [min]"] + 0.0001)

train["torque times wear"] = train["Torque [Nm]"] * train["Tool wear [min]"]
test["torque times wear"] = test["Torque [Nm]"] * test["Tool wear [min]"]

In [15]:
predictor = TabularPredictor(label='Machine failure', problem_type='binary',
                             eval_metric='roc_auc',sample_weight='balance_weight').fit(train,presets="best_quality",
                                                                    auto_stack=True,save_space=True,
                                                                    num_stack_levels=2, num_bag_folds=3,num_bag_sets=6,time_limit=36000)

# ['accuracy', 'acc', 'balanced_accuracy', 'mcc', 'roc_auc_ovo_macro', 'log_loss', 'nll', 'pac', 'pac_score', 'quadratic_kappa', 'roc_auc', 'average_precision', 'precision', 'precision_macro', 'precision_micro',
#  'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_weighted']

No path specified. Models will be saved in: "AutogluonModels/ag-20230615_171433/"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=2, num_bag_folds=3, num_bag_sets=6
Using predefined sample weighting strategy: balance_weight. Evaluation metrics will ignore sample weights, specify weight_evaluation=True to instead report weighted metrics.
Beginning AutoGluon training ... Time limit = 36000s
AutoGluon will save models to "AutogluonModels/ag-20230615_171433/"
AutoGluon Version:  0.7.0
Python Version:     3.10.10
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Fri Jun 9 10:57:30 UTC 2023
Train Data Rows:    145273
Train Data Columns: 18
Label Column: Machine failure
Preprocessing data ...
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Assigning sample weights to balance differences in frequency of classes.
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Av

In [16]:
preds = predictor.predict_proba(test)

In [17]:
preds.shape

(90954, 2)

In [18]:
!rm -rf /kaggle/working/*

In [19]:
preds=preds.iloc[: , 1]

* After fitting

In [20]:
preds = preds.apply(lambda x: 1 if x >= 0.995 else (0 if x <= 0.005 else x))

In [21]:
submission['Machine failure'] = preds

In [22]:
submission.describe()

,id,Machine failure
count,90954.000000,90954.000000
mean,181905.500000,0.013598
std,26256.302529,0.109335
min,136429.000000,0.000000
25%,159167.250000,0.000000
50%,181905.500000,0.000000
75%,204643.750000,0.000000
max,227382.000000,1.000000


In [23]:
submission.to_csv('/kaggle/working/submission.csv', index=False)